In [0]:
import torch


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import torchvision
import torchvision.transforms as transforms

# 불러온 이미지 선처리 하기 위한 단계 (ToTensor : 이미지를 텐서 형태로)
# Tensor의 range는 0에서 1로
# 채널에 변화를 줌(?)
# CenterCrop : 이미지 가운데 부분만 잘라서 사용하겠다
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
# 이미지 전처리를 위한 transforms.Compose
# PIL image (H x W x C) -> Tensor (C x H x W)

trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=False,
                                       transform=transform)

In [0]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, 
                         batch_size=4,
                         shuffle=True, 
                         drop_last=False)

testloader = DataLoader(testset, 
                        batch_size=4,
                        shuffle=False,
                        drop_last=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
trainiter = iter(trainloader)
images, labels = trainiter.next()

print(len(trainloader))
print(images.shape)
print(labels)

12500
torch.Size([4, 3, 32, 32])
tensor([5, 2, 4, 1])


In [0]:
src = {'input_size':3*32*32,
       'hidden_size1':256,
       'hidden_size2':128,
       'hidden_size3':64,
       'output_size':10,
       'init_weight_range':0.5,
       'num_epochs':5,
       'batch_size':4,
       'learning_rate':1e-3}

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class TobigsNet(nn.Module):
    def __init__(self, src):
        super(TobigsNet, self).__init__()
        self.fc1 = nn.Linear(src['input_size'], src['hidden_size1'])
        self.fc2 = nn.Linear(src['hidden_size1'], src['hidden_size2'])
        self.fc3 = nn.Linear(src['hidden_size2'], src['hidden_size3'])
        self.fc4 = nn.Linear(src['hidden_size3'], src['output_size'])
                             
        ## sequential layer
        self.seq_fc = nn.Sequential(
                            nn.Linear(src['input_size'], src['hidden_size1']),
                            nn.Linear(src['hidden_size1'], src['hidden_size2']),
                            nn.Linear(src['hidden_size2'], src['hidden_size3']),
                            nn.Linear(src['hidden_size3'], src['output_size'])
                            )
        
        self.init_range = src['init_weight_range']
        
    def init_weight(self):
        self.fc1.weight.data.uniform_(-self.init_range, self.init_range)
        self.fc2.weight.data.uniform_(-self.init_range, self.init_range)
        self.fc3.weight.data.uniform_(-self.init_range, self.init_range)
        self.fc4.weight.data.uniform_(-self.init_range, self.init_range)
        
        for fc in self.seq_fc:
            fc.weight.data.uniform_(-self.init_range, self.init_range)

    def forward(self, img):
        x = img.view(img.shape[0], -1)
        ## 야심차게 4-layer-NN
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        y= F.softmax(x,  dim=0)

        

        return y

In [0]:
model = TobigsNet(src)
y = model(images)

In [12]:
print(y.shape)

torch.Size([4, 10])


In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      src['learning_rate'], 
                      momentum=0.9)

In [10]:
from tqdm import tqdm

model.init_weight()

for epoch in tqdm(range(src['num_epochs'])):
    current_loss = 0.0
#     model.train(True)
    
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        step = i + 1
        current_loss += loss.item()
        
        if step % 1000 == 0 and step != 0:     # print every 1000 mini-batches
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %
                  (epoch + 1, src['num_epochs'], step, len(trainloader)//1000 * 1000, current_loss / 1000))
            current_loss = 0.0

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [1/5], Step [1000/12000], Loss: 2.3205
Epoch [1/5], Step [2000/12000], Loss: 2.2579
Epoch [1/5], Step [3000/12000], Loss: 2.2478
Epoch [1/5], Step [4000/12000], Loss: 2.2359
Epoch [1/5], Step [5000/12000], Loss: 2.2323
Epoch [1/5], Step [6000/12000], Loss: 2.2348
Epoch [1/5], Step [7000/12000], Loss: 2.2333
Epoch [1/5], Step [8000/12000], Loss: 2.2283
Epoch [1/5], Step [9000/12000], Loss: 2.2269
Epoch [1/5], Step [10000/12000], Loss: 2.2142
Epoch [1/5], Step [11000/12000], Loss: 2.2122
Epoch [1/5], Step [12000/12000], Loss: 2.2018


 20%|██        | 1/5 [01:31<06:07, 91.77s/it]

Epoch [2/5], Step [1000/12000], Loss: 2.1974
Epoch [2/5], Step [2000/12000], Loss: 2.2001
Epoch [2/5], Step [3000/12000], Loss: 2.2036
Epoch [2/5], Step [4000/12000], Loss: 2.1995
Epoch [2/5], Step [5000/12000], Loss: 2.2058
Epoch [2/5], Step [6000/12000], Loss: 2.2140
Epoch [2/5], Step [7000/12000], Loss: 2.2087
Epoch [2/5], Step [8000/12000], Loss: 2.2114
Epoch [2/5], Step [9000/12000], Loss: 2.1949
Epoch [2/5], Step [10000/12000], Loss: 2.1977
Epoch [2/5], Step [11000/12000], Loss: 2.1828
Epoch [2/5], Step [12000/12000], Loss: 2.1961


 40%|████      | 2/5 [03:04<04:35, 91.97s/it]

...

 60%|██████    | 3/5 [04:34<03:03, 91.61s/it]

Epoch [4/5], Step [1000/12000], Loss: 2.1953
Epoch [4/5], Step [2000/12000], Loss: 2.1995
Epoch [4/5], Step [3000/12000], Loss: 2.1795
Epoch [4/5], Step [4000/12000], Loss: 2.1941
Epoch [4/5], Step [5000/12000], Loss: 2.1971
Epoch [4/5], Step [6000/12000], Loss: 2.1750
Epoch [4/5], Step [7000/12000], Loss: 2.1608
Epoch [4/5], Step [8000/12000], Loss: 2.2061
Epoch [4/5], Step [9000/12000], Loss: 2.2100
Epoch [4/5], Step [10000/12000], Loss: 2.2017
Epoch [4/5], Step [11000/12000], Loss: 2.1858
Epoch [4/5], Step [12000/12000], Loss: 2.1809


 80%|████████  | 4/5 [06:08<01:32, 92.23s/it]

Epoch [5/5], Step [1000/12000], Loss: 2.1898
Epoch [5/5], Step [2000/12000], Loss: 2.1882
Epoch [5/5], Step [3000/12000], Loss: 2.1963
Epoch [5/5], Step [4000/12000], Loss: 2.2004
Epoch [5/5], Step [5000/12000], Loss: 2.1942
Epoch [5/5], Step [6000/12000], Loss: 2.1884
Epoch [5/5], Step [7000/12000], Loss: 2.1912
Epoch [5/5], Step [8000/12000], Loss: 2.1770
Epoch [5/5], Step [9000/12000], Loss: 2.1823
Epoch [5/5], Step [10000/12000], Loss: 2.1885
Epoch [5/5], Step [11000/12000], Loss: 2.1910
Epoch [5/5], Step [12000/12000], Loss: 2.1905


100%|██████████| 5/5 [07:39<00:00, 91.73s/it]


In [11]:
# Test the Model
correct = 0
total = 0
for i, data in enumerate(testloader):
    inputs, labels = data
#     images = images.view(-1, 28*28)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.shape[0]
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))

#Accuracy of the network on the 2500 test images: 17 %
#ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ;;;;;;

Accuracy of the network on the 2500 test images: 17 %
